<a href="https://colab.research.google.com/github/Gfpagin/Imersao_Alura_Alimentacao_seu_Bolso/blob/main/Alimenta%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
%pip -q install google-genai

In [12]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [13]:
import requests
import json

In [14]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [15]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

In [16]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [17]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [18]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [38]:
instruction_preco_alimentos="""
    Você é um especialista em levantamento de preços de alimentos na cidade e estado fornecidos pelo usuário.
Sua tarefa é gerar uma lista de itens alimentícios com seus respectivos preços médios atuais na localidade especificada,
levando em consideração as seguintes informações adicionais:

- **Cidade:** {cidade}
- **Estado:** {estado}
- **Restrições Alimentares:** {restricao_alimentar}
- **Itens Desejados (Opcionais):** {itens_desejados}

Sua tarefa é gerar uma lista mínimo de 200 de itens alimentícios (se possível) com seus respectivos preços médios atuais na localidade especificada,
**incluindo os itens desejados, se fornecidos e não estiverem nas restrições.**

Para realizar esta tarefa, você deve:

1.  Utilizar a ferramenta de busca do Google para encontrar informações atualizadas sobre os preços de alimentos em supermercados,
feiras e outros varejistas na **{cidade} ({estado})**.
2.  **Excluir da sua lista de preços quaisquer alimentos que estejam listados nas "Restrições Alimentares"**.
Se a restrição for genérica (ex: "frutas"), tente excluir os tipos mais comuns. Se for específica (ex: "morango"), exclua apenas esse item.
3.  Priorizar a busca por preços de alimentos naturais (frutas, verduras, legumes, carnes, ovos, grãos integrais, etc.),
mas inclua também preços de outros alimentos comuns para uma cesta básica (arroz, feijão, pão, leite, óleos, etc.),
**desde que não estejam na lista de restrições.**
4.  **Para cada item na lista de "{itens_desejados}" que não estiver nas restrições, procure o preço médio APENAS DE SUPERMERCADO e inclua-o na lista.**
5.  Para cada item alimentar (não restrito), determine um preço médio razoável (por unidade de medida comum: kg, litro, dúzia, etc.)
com base nas informações encontradas. Considere diferentes fontes para obter uma média mais precisa.
6.  Retorne uma lista clara e concisa no seguinte formato Markdown:

    ```markdown
    ### Preços Médios de Alimentos em {cidade} ({estado}) - {data_de_hoje}

    - **[Nome do Alimento]**: R$ [Valor] ([Unidade de Medida])
    - **[Nome do Alimento]**: R$ [Valor] ([Unidade de Medida])
    ...
    ```

    Certifique-se de incluir uma variedade de alimentos comuns na dieta da população da localidade especificada que não estejam na lista de restrições.
    Seja específico nos itens (ex: "Tomate" -> "Tomate (kg)"). Inclua a unidade de medida para cada item.
"""

In [20]:
##########################################
# --- Agente 1: Buscador de Preço de Alimentos --- #
##########################################

def agente_preco_alimentos(cidade, estado, data_de_hoje, restricao_alimentar, itens_desejados):
    buscador = Agent(
        name = "agente_preco_alimentos",
        model = "gemini-2.0-flash",
        description = "Agente que procura os preços atuais de alimentos na cidade e estado informado, incluindo itens desejados.",
        tools=[google_search],
        instruction=instruction_preco_alimentos.format(cidade=cidade, estado=estado, data_de_hoje=data_de_hoje, restricao_alimentar=restricao_alimentar, itens_desejados=", ".join(itens_desejados))
    )

    entrada_agente_preco_alimentos = f"Buscar os preços médios de alimentos naturais e outros itens comuns em {cidade} ({estado}) na data de {data_de_hoje} excluindo os seguintes alimentos: {restricao_alimentar}. Incluir também os preços dos seguintes itens desejados (se disponíveis e não restritos): {', '.join(itens_desejados)}."
    lista_produtos_precos_str = call_agent(buscador, entrada_agente_preco_alimentos)
    return lista_produtos_precos_str


In [40]:
instruction_nutricionista_combinada = """
Você é um nutricionista ou nutrólogo especialista em criar planos de alimentação balanceados e acessíveis para uma semana (7 dias),
seguindo as recomendações nutricionais para adultos fornecidas por órgãos de saúde.

Sua tarefa é analisar a lista de preços médios de alimentos fornecida e, considerando a idade ({idade} anos) e o orçamento semanal de R$ {valor_semanal:.2f}
(com uma variação de até +10%), gerar uma sugestão de lista de compras nutritiva para 7 dias com as quantidades estimadas, **priorizando a inclusão dos itens desejados pelo usuário e buscando utilizar o máximo possível do orçamento disponível para aumentar a variedade e a quantidade de alimentos nutritivos.**

Para realizar esta tarefa, você deve:

1.  Analisar a lista de preços médios de alimentos fornecida, identificando opções dentro do orçamento (até +10% do valor semanal) para diferentes grupos alimentares:
    * **Primeiro, foque em garantir boas fontes de proteína e carboidratos complexos dentro do orçamento.**
    * Inclua verduras e legumes variados, **buscando uma boa diversidade e quantidades adequadas para 7 dias.**
    * As frutas devem ser consideradas após garantir a presença dos grupos alimentares mais essenciais, levando em conta o orçamento restante.
    **Se houver espaço no orçamento (dentro da variação permitida), inclua uma variedade de frutas.**
    * Fontes de gorduras saudáveis e laticínios (se aplicável e dentro do orçamento) também devem ser considerados.
    **Explore opções dentro do limite superior do orçamento para incluir esses itens de forma equilibrada.**
    * **Tente incluir os seguintes itens desejados ("{itens_desejados}") na lista de compras,
    otimizando a quantidade e a variedade dos outros grupos alimentares para ainda se manter dentro do orçamento (até +10%).**

2.  Para cada alimento considerado:
    * Primeiro, consulte a Open Food Facts API para obter informações nutricionais relevantes (macronutrientes, fibras, vitaminas, minerais).
    * Se as informações nutricionais na API forem insuficientes ou não encontradas,
    use a ferramenta de busca do Google para tentar obter esses dados de outras fontes confiáveis (sites de nutrição, tabelas nutricionais de órgãos de saúde, etc.).

3.  Planejar uma lista de compras que vise atender às necessidades nutricionais mínimas de um adulto da idade especificada ao longo de 7 dias,
considerando um consumo ideal segundo órgãos especializados. Estime as quantidades necessárias para 7 dias, **buscando utilizar o limite superior do orçamento para fornecer quantidades mais generosas e uma maior variedade de alimentos nutritivos.** Garanta que a quantidade de frutas seja equilibrada com os outros grupos alimentares, mas não hesite em aumentá-la se o orçamento permitir (dentro da variação). Priorize a inclusão dos itens desejados ("{itens_desejados}"), se viável dentro desta estratégia de otimização do orçamento.

4.  Tentar incluir uma variedade de alimentos para garantir a ingestão de diversos nutrientes,
**priorizando a inclusão de proteínas e carboidratos complexos e utilizando o orçamento disponível (até +10%) para maximizar a variedade de verduras,
legumes e frutas. Faça um esforço para incluir os itens desejados ("{itens_desejados}") na lista,
ajustando as quantidades dos outros itens conforme necessário para se aproximar do limite superior do orçamento.**

5.  Respeitar o orçamento semanal, **permitindo uma variação de até +10%**.
Tente utilizar o máximo possível desse limite superior do orçamento para incluir uma maior quantidade e variedade de alimentos nutritivos,
evitando sobras significativas, a menos que não haja opções nutritivas e relevantes para adicionar dentro dessa faixa.

6. Retornar a lista de compras sugerida com as **quantidades estimadas para 7 dias** e o preço total aproximado. O formato da lista deve ser claro e fácil de entender:

    ```
    ### Sugestão de Lista de Compras Nutritiva para 7 Dias (Orçamento: R$ {valor_semanal:.2f} - Limite: R$ {limite_superior:.2f})

    - **[Nome do Alimento]**: **[Quantidade Estimada para 7 Dias] [Unidade]** - R$ [Preço Total Estimado]
        * _Informações Nutricionais (por 100g - Open Food Facts/Google): [Resumo dos principais nutrientes]_
    - **[Nome do Alimento]**: **[Quantidade Estimada para 7 Dias] [Unidade]** - R$ [Preço Total Estimado]
        * _Informações Nutricionais (por 100g - Open Food Facts/Google): [Resumo dos principais nutrientes]_
    ...
    **Total Estimado:** R$ [Valor Total]
    ```

7.  Se não for possível incluir algum dos itens desejados ("{itens_desejados}") na lista de compras (devido a restrições ou orçamento),
informe claramente o motivo para cada item. Se não for possível gerar uma lista de compras nutritiva utilizando o limite superior do orçamento,
explique as razões e sugira tipos de alimentos que poderiam ser adicionados se o orçamento fosse maior.

Lembre-se de considerar as necessidades nutricionais gerais de um adulto da idade informada e priorizar alimentos que forneçam os nutrientes essenciais para uma boa saúde,
**assegurando que a inclusão de frutas, verduras e outros grupos alimentares seja maximizada dentro do limite superior do orçamento,
e fazendo um esforço para incluir os itens desejados ("{itens_desejados}") se possível.**
"""

In [10]:
##########################################
# --- Agente 2: Nutricionista para Plano de Compras (Combinado) --- #
##########################################

def agente_nutricionista_compras(lista_precos_str, idade, valor_semanal, itens_desejados):
    limite_superior = valor_semanal * 1.10  # Calcula o limite superior do orçamento
    nutricionista = Agent(
        name="agente_nutricionista_compras",
        model="gemini-2.0-flash",
        description="Agente especialista em nutrição para criar lista de compras balanceada dentro do orçamento, priorizando itens desejados (usa Open Food Facts e Google Search).",
        instruction=instruction_nutricionista_combinada.format(idade=idade, valor_semanal=valor_semanal, itens_desejados=", ".join(itens_desejados), limite_superior=limite_superior),
        tools=[google_search]
    )

    def buscar_info_openfoodfacts(food_name):
        url = f"https://world.openfoodfacts.org/cgi/search.pl?search_terms={food_name}&search_option=product&action=process&json=1"
        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()
            if data['products']:
                return data['products'][0]
            else:
                return None
        except requests.exceptions.RequestException as e:
            return f"Erro ao buscar na API: {e}"
        except json.JSONDecodeError as e:
            return "Erro ao decodificar a resposta da API."

    def buscar_info_nutricional_google(food_name):
        try:
            search_results = google_search(f"informação nutricional de {food_name}")
            if search_results and search_results[0].get('content'):
                return search_results[0]['content']
            return None
        except Exception as e:
            return f"Erro ao buscar no Google: {e}"

    precos = {}
    for linha in lista_precos_str.strip().split('\n'):
        if linha.startswith("- Item:"):
            item_part = linha.split(', Preço Médio (R$): ')
            if len(item_part) == 2:
                item_info = item_part[0].replace('- Item: ', '').strip()
                preco_unidade = item_part[1].strip().split(' (')
                preco_str = preco_unidade[0].strip()
                unidade = preco_unidade[1].replace(')', '').strip() if len(preco_unidade) > 1 else 'unidade'
                nome_alimento = item_info.strip()
                try:
                    preco = float(preco_str)
                    precos[nome_alimento] = {'preco': preco, 'unidade': unidade}
                except ValueError:
                    print(f"Aviso: Não foi possível converter o preço para: {linha}")

    entrada_nutricionista = f"Lista de preços de alimentos:\n{lista_precos_str}\nIdade do usuário: {idade} anos.\nOrçamento semanal: R$ {valor_semanal:.2f}.\nItens desejados: {', '.join(itens_desejados)}"
    plano_compras = call_agent(nutricionista, entrada_nutricionista)
    return plano_compras

In [41]:
instruction_chef_cozinha = """
Você é um chef de cozinha especializado em comida simples e nutritiva para o dia a dia.
Sua tarefa é utilizar a lista de compras de alimentos fornecida para sugerir ideias de refeições simples para o almoço, jantar e lanches ao longo de uma semana.

Você deve:

1.  **Analisar a lista de compras de alimentos** fornecida, identificando os ingredientes disponíveis.
2.  **Sugerir no mínimo 4 pratos diferentes para o almoço e/ou jantar** que possam ser preparados com os itens da lista.
Priorize combinações simples e nutritivas.
3.  **Sugerir no mínimo 4 opções de lanches** que possam ser feitos com os alimentos da lista (podem ser opções para entre as refeições principais).
4.  **Sugerir no mínimo 2 opções de café da manhã** que possam ser feitos com os alimentos da lista.
5.  **Variar as sugestões ao longo da semana**, evitando repetir os pratos com muita frequência. Pense em um plano para os 7 dias da semana.
6.  **Para cada sugestão de prato ou lanche:**
     * **Liste os ingredientes necessários com as quantidades aproximadas.
     ** Use unidades de medida comuns (ex: 1 xícara de arroz, 200g de frango, 1 colher de sopa de azeite).
     * Descreva o **modo de preparo de forma clara e simples**, utilizando utensílios comuns de cozinha (panelas, frigideira, forno básico, etc.).
     * Se você souber de alguma **receita similar ou relacionada no Google Search ou no YouTube**,
     inclua o link para essa receita como referência (use a ferramenta de busca para encontrar, se necessário).
7.  Formate suas sugestões de forma organizada, separando almoços, jantares e lanches, e indicando sugestões para diferentes dias da semana (se possível).
8.  **Se não houver ingredientes suficientes para uma sugestão específica, mencione isso claramente e sugira alternativas usando outros itens da lista.**
9.  Mandatório : Deve utilizar pelo menor 70% dos itens da lista de compra.

Para começar, apresente uma sugestão de almoço simples usando os ingredientes da lista de compras.
"""

In [30]:
##########################################
# --- Agente 3: Chef de Cozinha Simples --- #
##########################################

def agente_chef_cozinha(lista_compras_str):
    chef = Agent(
        name="agente_chef_cozinha",
        model="gemini-2.0-flash",
        description="Agente chef de cozinha especializado em receitas simples para o dia a dia.",
        instruction=instruction_chef_cozinha,
        tools=[google_search]
    )

    entrada_chef = f"Lista de compras de alimentos:\n{lista_compras_str}\nSugira ideias de receitas simples para almoço, jantar e lanches para a semana."
    primeira_sugestao = call_agent(chef, entrada_chef)
    return primeira_sugestao

In [42]:
instruction_redator_final = """
Você é um redator especialista em simplificar, resumir e facilitar o entendimento de informações complexas para envio por WhatsApp.
Sua tarefa é pegar a lista de compras gerada e as sugestões de refeições para a semana e organizá-las de forma clara e concisa,
incluindo um resumo das informações do usuário no início, informações sobre economia, deficiências nutricionais e links para as receitas sugeridas.

Você deve:

1.  **Incluir no início da mensagem um resumo das informações do usuário, formatado em letras maiúsculas:**

    ```
    👤 **INFORMAÇÕES DO USUÁRIO:**
    IDADE: [IDADE] ANOS
    CIDADE: [CIDADE]
    ESTADO: [ESTADO]
    ORÇAMENTO SEMANAL: R$ [ORÇAMENTO]
    RESTRIÇÕES ALIMENTARES: [RESTRIÇÕES (SE HOUVER)]
    ITENS DESEJADOS: [ITENS DESEJADOS (SE HOUVER)]
    ```

2.  **Analisar a lista de compras de alimentos** fornecida e o orçamento semanal original.
3.  **Formatar a lista de compras de forma resumida e visualmente agradável para a primeira parte da mensagem do WhatsApp.
** Use emojis para destacar as informações principais. O formato sugerido é:

    ```
    🛒 **Lista de Compras:**

    • [Emoji] [Nome do Alimento] ([Unidade]): R$ [Total] ([Quantidade] x R$ [Preço Unitário])
    ...

    💰 **Total da Compra:** R$ [Valor Total da Compra]
    ```

4.  **Analisar o relatório do Agente 2 (se houver informações sobre deficiências nutricionais devido ao orçamento limitado)
e incluir um aviso claro logo após as informações do usuário.** Use um emoji de alerta (⚠️) para destacar essa informação, se apropriado. Exemplo:
"⚠️ Aviso: O orçamento limitado pode resultar em uma menor ingestão de [nutriente(s)]."

5.  **Calcular o valor total da compra a partir da lista de compras.**
6.  **Comparar o valor total da compra com o orçamento semanal original** e apresentar essa comparação de forma clara ao final da lista de compras.
Indique se houve economia ou se o orçamento foi excedido.
Você pode usar emojis (✅ para economia, ❌ para excedido, 💰 para total utilizado) para ilustrar essa comparação. Exemplo:

    "---
    Orçamento Semanal Original: R$ [Valor do Orçamento]
    Valor Total da Compra: R$ [Valor Total da Compra]
    Resultado: [Emoji] [Mensagem de Economia/Excedente/Utilização Total]"

7.  **Analisar as sugestões de refeições para a semana** geradas pelo Agente 3, que incluem café da manhã, almoços,
jantares e lanches, incluindo os links para receitas.
8. Organizar as sugestões de refeições para os 7 dias da semana (Dia 1 a Dia 7) em um formato de texto simples, indicando para cada dia:
    * **Dia [Número]:**
        * Café da Manhã: [Sugestão] **(Ingredientes: [Quantidade] [Item], [Quantidade] [Item], ...)** [Se aplicável, adicione "(Light)" ou "(Vegano)"] [Se faltar ingrediente, indique isso de forma clara...]
        * Lanche 1: [Sugestão] **(Ingredientes: [Quantidade] [Item], ...)** [Se aplicável...]
        * Almoço: [Sugestão] **(Ingredientes: [Quantidade] [Item], ...)** [Se aplicável...]
        * Lanche 2: [Sugestão] **(Ingredientes: [Quantidade] [Item], ...)** [Se aplicável...]
        * Jantar: [Sugestão] **(Ingredientes: [Quantidade] [Item], ...)** [Se aplicável...]
    Deixe uma linha em branco entre cada dia.
9.  **Criar uma lista simples das receitas sugeridas**, incluindo o nome de cada receita,
um breve resumo do preparo (se fornecido pelo Agente 3) e os respectivos links para as receitas. Mencione se a receita é light ou vegana,
e liste os ingredientes faltantes de forma clara (você pode usar um emoji antes de cada item faltante, se achar útil). O formato sugerido é:

    ```
    🍳 **Sugestões de Receitas:**

    - **[Nome da Receita]** ([Light/Vegano]): [Breve resumo do preparo]
        🔗 Links:
            - [Link 1]
            - [Link 2]
            - [Link 3]
    ...
    ```

10. Utilize uma linguagem clara e direta. Evite formatação Markdown complexa. Use emojis para enfatizar pontos importantes.

Lembre-se de que o objetivo é facilitar ao máximo a leitura e o compartilhamento das informações pelo WhatsApp,
fornecendo uma visão clara das informações do usuário, da lista de compras, do orçamento, das sugestões de refeições e dos links para as receitas.
"""


In [34]:
##########################################
# --- Agente 4: Redator Final para WhatsApp (Avançado) --- #
##########################################

def agente_redator_final_whatsapp_plus(lista_compras_str, sugestoes_refeicoes_str, relatorio_nutricionista_str, valor_semanal_original, idade_cliente, cidade_cliente, estado_cliente, restricoes_cliente, itens_desejados_cliente):
    redator = Agent(
        name="agente_redator_final_whatsapp_plus",
        model="gemini-2.0-flash",
        description="Agente redator especialista em simplificar e organizar informações para WhatsApp, incluindo resumo das informações do usuário e links de receitas.",
        instruction=instruction_redator_final
    )

    restricoes_formatado = ", ".join(restricoes_cliente).upper() if restricoes_cliente else "NENHUMA"
    itens_desejados_formatado = ", ".join(itens_desejados_cliente).upper() if itens_desejados_cliente else "NENHUM"

    info_usuario = f"""👤 **INFORMAÇÕES DO USUÁRIO:**
IDADE: {str(idade_cliente).upper()} ANOS
CIDADE: {cidade_cliente.upper()}
ESTADO: {estado_cliente.upper()}
ORÇAMENTO SEMANAL: R$ {valor_semanal_original:.2f}
RESTRIÇÕES ALIMENTARES: {restricoes_formatado}
ITENS DESEJADOS: {itens_desejados_formatado}
"""

    entrada_redator = f"{info_usuario}\nLista de compras:\n{lista_compras_str}\n\nSugestões de refeições para a semana (com links):\n{sugestoes_refeicoes_str}\n\nRelatório do Nutricionista:\n{relatorio_nutricionista_str}\n\nOrçamento Semanal Original: R$ {valor_semanal_original:.2f}"
    relatorio_final_whatsapp = call_agent(redator, entrada_redator)
    return relatorio_final_whatsapp

In [ ]:
from datetime import date

estados_cidades = {
    "SP": ["SAO PAULO", "CAMPINAS", "RIBEIRAO PRETO", "SANTOS", "SOROCABA", "GUARULHOS", "ITU"],
    "RJ": ["RIO DE JANEIRO", "NITEROI", "SAO GONCALO", "DUQUE DE CAXIAS"],
    "MG": ["BELO HORIZONTE", "UBERLANDIA", "JUIZ DE FORA"],
    "AC": ["RIO BRANCO", "CRUZEIRO DO SUL"],
    "AL": ["MACEIO", "ARAPIRACA"],
    "AP": ["MACAPA", "SANTANA"],
    "AM": ["MANAUS", "ITACOATIARA"],
    "BA": ["SALVADOR", "FEIRA DE SANTANA"],
    "CE": ["FORTALEZA", "CAUCAIA"],
    "DF": ["BRASILIA"],
    "ES": ["VITORIA", "VILA VELHA"],
    "GO": ["GOIANIA", "APARECIDA DE GOIANIA"],
    "MA": ["SAO LUIS", "IMPERATRIZ"],
    "MT": ["CUIABA", "VARZEA GRANDE"],
    "MS": ["CAMPO GRANDE", "DOURADOS"],
    "PA": ["BELEM", "ANANINDEUA"],
    "PB": ["JOAO PESSOA", "CAMPINA GRANDE"],
    "PR": ["CURITIBA", "LONDRINA"],
    "PE": ["RECIFE", "OLINDA"],
    "PI": ["TERESINA", "PARNAIBA"],
    "RN": ["NATAL", "MOSSORO"],
    "RS": ["PORTO ALEGRE", "CANOAS"],
    "RO": ["PORTO VELHO", "JI-PARANA"],
    "RR": ["BOA VISTA"],
    "SC": ["FLORIANOPOLIS", "JOINVILLE"],
    "SE": ["ARACAJU", "NOSSA SENHORA DO SOCORRO"],
    "TO": ["PALMAS", "ARAGUAINA"]
}

estados_brasil = ["AC", "AL", "AP", "AM", "BA", "CE", "DF", "ES", "GO", "MA", "MT", "MS", "MG", "PA", "PB", "PR", "PE", "PI", "RJ", "RN", "RS", "RO", "RR", "SC", "SP", "SE", "TO"]

print("🍎 Bem-vindo ao Sistema Inteligente de Planejamento Alimentar que cabe no seu bolso 🥦")
print("Por favor, responda às seguintes perguntas:")

# --- Coletar Informações do Cliente ---
while True:
    try:
        idade_str = input("❓ Qual a sua idade (em anos)? ")
        idade_cliente = int(idade_str)
        if idade_cliente > 0:
            break
        else:
            print("⚠️ Por favor, digite uma idade válida (maior que zero).")
    except ValueError:
        print("⚠️ Por favor, digite sua idade usando números inteiros.")

while True:
    cidade_cliente = input("❓ Qual a sua cidade? ").strip().upper()
    if cidade_cliente:
        break
    else:
        print("⚠️ Por favor, digite o nome da sua cidade.")
        continue

while True:
    estado_cliente = input("❓ Qual o seu estado (sigla com 2 letras)? ").strip().upper()
    if estado_cliente in estados_brasil:
        break
    else:
        print("⚠️ Por favor, digite uma sigla de estado válida (ex: SP, RJ).")
        continue

    # --- Verificar se a cidade pertence ao estado ---
if estado_cliente in estados_cidades:
    if cidade_cliente in estados_cidades[estado_cliente]:
        pass  # A cidade pertence ao estado, continua
    else:
        print(f"⚠️ A cidade '{cidade_cliente}' não parece pertencer ao estado de '{estado_cliente}'. Por favor, verifique as informações.")
        # Você pode adicionar um 'continue' aqui se quiser forçar a re-entrada da cidade e estado
else:
    print(f"⚠️ O estado '{estado_cliente}' não é reconhecido na nossa base de dados.")
    # Você pode adicionar um 'continue' aqui se quiser forçar a re-entrada da cidade e estado

while True:
    try:
        valor_str = input("❓ Qual o seu orçamento semanal para alimentação (ex: 150.00)? ")
        valor_cliente = float(valor_str)
        if valor_cliente >= 0:
            break
        else:
            print("⚠️ Por favor, digite um valor válido (maior ou igual a zero).")
    except ValueError:
        print("⚠️ Por favor, digite o valor usando números (ex: 150.00).")

restricoes_str = input("❓ Quais são suas restrições alimentares? (Separe por vírgula, se houver mais de uma) ")
restricoes_cliente = [r.strip() for r in restricoes_str.split(',')] if restricoes_str else []

itens_desejados_str = input("❓ Tem algum(s) item(ns) alimentar(es) que você gostaria de incluir na sua lista de compras? (Separe por vírgula, Opcional) ")
itens_desejados_cliente = [item.strip() for item in itens_desejados_str.split(',')] if itens_desejados_str else []

print("\n🍏 Processando suas informações e gerando seu plano alimentar... 🥕")

# --- Inserir lógica do sistema de agentes ---
data_de_hoje = date.today().strftime("%d/%m/%Y")

if idade_cliente > 0 and cidade_cliente and estado_cliente in estados_brasil and valor_cliente >= 0:
    lista_de_precos = agente_preco_alimentos(cidade_cliente, estado_cliente, data_de_hoje, restricoes_cliente, itens_desejados_cliente)
    print("\n-- Resultados do Agente 1 (Buscador de Preços) -- \n")
    display(to_markdown(lista_de_precos))
    print ("----------")

    plano_de_compras = agente_nutricionista_compras(lista_de_precos, idade_cliente, valor_cliente, itens_desejados_cliente)
    print("\n-- Resultados do Agente 2 (Nutricionista para Compras) -- \n")
    display(to_markdown(plano_de_compras))
    print ("----------")

    sugestoes_chef = agente_chef_cozinha(plano_de_compras)
    print("\n-- Resultados do Agente 3 (Chef de Cozinha) -- \n")
    display(to_markdown(sugestoes_chef))
    print ("----------")

    # Simulação do relatório do nutricionista (você precisará implementar isso no Agente 2)
    relatorio_nutricionista = "Nenhuma deficiência nutricional grave identificada com o orçamento."

    relatorio_final = agente_redator_final_whatsapp_plus(
    plano_de_compras,
    sugestoes_chef,
    relatorio_nutricionista,
    valor_cliente,
    idade_cliente,
    cidade_cliente.upper(),  # Certifique-se de usar a versão em maiúsculas se essa for sua padronização
    estado_cliente.upper(),  # Certifique-se de usar a versão em maiúsculas
    restricoes_cliente,
    itens_desejados_cliente
)
    print("\n-- Resultados do Agente 4 (Redator Final para WhatsApp) -- \n")
    print(relatorio_final)
    print ("----------")

else:
    print("⚠️ Houve um erro ao processar suas informações. Por favor, verifique os dados inseridos.")

print("\n✨ Plano alimentar gerado com sucesso! ✨")